Step 1: Importing + Reading data

In [1]:
import json
import re
from collections import Counter
import numpy as np
import pyspark
import os
from datetime import datetime
from functools import partial

from pyspark.sql.functions import year
from pyspark.sql.functions import month
from pyspark.sql.functions import dayofmonth


from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
from pyspark.sql import functions as F

import webbrowser
import mysql.connector

In [2]:
index2reviews_column = {
    6: 'product_category',
    1: 'customer_id',
    14: 'review_date',
    7: 'star_rating'   
}
amazon_data = sc.textFile("amazon/amazon_reviews_multilingual_US_v1_00.tsv")
tab_split_func = lambda input_line : input_line.split("\t")

In [3]:
amazon_split_rdd = amazon_data.map(tab_split_func)

In [4]:
"check"
#amazon_data.take(2)

'check'

In [5]:
"check"
#amazon_split_rdd.take(2)

'check'

In [6]:
def transform_to_json(record, index2column):
    return_value = dict()
    for index, column_name in index2column.items():
        return_value[column_name] = record[index]
    return return_value

amazon_json_rdd = amazon_split_rdd.map(partial(transform_to_json, index2column=index2reviews_column))

In [7]:
"check"
#amazon_json_rdd.take(2)

'check'

In [8]:
"checks"
#amazon_json_rdd.cache()
#amazon_json_rdd.zipWithIndex()
#amazon_json_rdd.take(2)

'checks'

In [9]:
"ignore"
#amazon_filter_json_rdd = amazon_json_rdd.filter(_._2 is not 0)

'ignore'

In [10]:
#def extract_month(dtstr):
 #   format_string = dtstr.split("-")
  #  dt = format_string[1]
   # return dt

#def extract_year(dtstr):
 #   format_string = dtstr.split("-")
    #dt = format_string[0]
    #return dt

def extract_yearmonth(dtstr):
    format_string = dtstr.split("-")
    dt =  format_string[0]+ "-" +format_string[1]
    return dt

def convert_dtstr_dt(dtstr):
    format_string = "%Y-%m-%d"
    dt = datetime.strptime(dtstr, format_string)
    return dt



type_converters = {
#might need further additions
    "review_date" : extract_yearmonth
}


def convert_types(record, converters):
    for col_name, convert_func in converters.items():
        record[col_name] = convert_func(record[col_name])
    return record

In [11]:
assert convert_dtstr_dt("2020-03-10") == datetime(2020, 3, 10)

In [12]:
assert extract_yearmonth("2020-03-12") == "2020-03"

In [13]:
"ignore"
#6931167

'ignore'

In [14]:
amazon_convert_prep_rdd = amazon_json_rdd.filter(lambda js : not js["customer_id"] == "customer_id")

In [15]:
amazon_convert_prep_rdd.take(2)

[{'product_category': 'Books',
  'customer_id': '53096384',
  'review_date': '1995-08-13',
  'star_rating': '4'},
 {'product_category': 'Books',
  'customer_id': '53096399',
  'review_date': '1995-08-17',
  'star_rating': '4'}]

In [16]:
amazon_converted_rdd = amazon_convert_prep_rdd.map(partial(convert_types, converters=type_converters))

In [17]:
"check"
amazon_converted_rdd.take(1)

[{'product_category': 'Books',
  'customer_id': '53096384',
  'review_date': '1995-08',
  'star_rating': '4'}]

In [18]:
"check"
amazon_converted_rdd.count()

6931166

In [19]:
unique_review_product_category_count =  amazon_converted_rdd.map(lambda json_record: (json_record["product_category"],1)).reduceByKey(lambda x, y: x+y)
unique_review_product_category_count.take(5)

[('Books', 838801),
 ('Toys', 57777),
 ('Pet Products', 5),
 ('Home Improvement', 3731),
 ('Grocery', 18)]

In [20]:
unique_review_per_product_category_rdd =  amazon_converted_rdd.map(lambda json_record: (json_record["product_category"],json_record))

In [21]:
joined_rdd = unique_review_per_product_category_rdd.join(unique_review_product_category_count)

In [22]:
unique_review_per_product_category_rdd.take(3)

[('Books',
  {'product_category': 'Books',
   'customer_id': '53096384',
   'review_date': '1995-08',
   'star_rating': '4'}),
 ('Books',
  {'product_category': 'Books',
   'customer_id': '53096399',
   'review_date': '1995-08',
   'star_rating': '4'}),
 ('Books',
  {'product_category': 'Books',
   'customer_id': '53096332',
   'review_date': '1995-08',
   'star_rating': '5'})]

In [23]:
joined_rdd.take(3)

[('Pet Products',
  ({'product_category': 'Pet Products',
    'customer_id': '18197812',
    'review_date': '2014-09',
    'star_rating': '5'},
   5)),
 ('Pet Products',
  ({'product_category': 'Pet Products',
    'customer_id': '29425839',
    'review_date': '2015-01',
    'star_rating': '5'},
   5)),
 ('Pet Products',
  ({'product_category': 'Pet Products',
    'customer_id': '37551413',
    'review_date': '2015-01',
    'star_rating': '4'},
   5))]

In [24]:
def add_reviews(tpl):
    product_category, (review_record, number_of_reviews) = tpl
    review_record["reviews"] = number_of_reviews
    return review_record

In [25]:
reviews_rdd = joined_rdd.map(add_reviews)

In [26]:
reviews_rdd.take(5)

[{'product_category': 'Pet Products',
  'customer_id': '18197812',
  'review_date': '2014-09',
  'star_rating': '5',
  'reviews': 5},
 {'product_category': 'Pet Products',
  'customer_id': '29425839',
  'review_date': '2015-01',
  'star_rating': '5',
  'reviews': 5},
 {'product_category': 'Pet Products',
  'customer_id': '37551413',
  'review_date': '2015-01',
  'star_rating': '4',
  'reviews': 5},
 {'product_category': 'Pet Products',
  'customer_id': '813252',
  'review_date': '2015-02',
  'star_rating': '3',
  'reviews': 5},
 {'product_category': 'Pet Products',
  'customer_id': '41598274',
  'review_date': '2015-08',
  'star_rating': '4',
  'reviews': 5}]